# Canada Neighborhoods

### Add imports necessary to work

In [2]:
###IMPORTS

#pip install beautifulsoup4
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import codecs
import pandas as pd

### Get Data

In [3]:
##GET SITE
response = urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
html = response.read()

In [4]:
##GET TABLE
soup = BeautifulSoup(html)
tabulka = soup.find("table", {"class" : "wikitable sortable"})

### Clean data

In [5]:
##SET VARIABLES
matrix = []
count = 0

##CUT DATA
for row in tabulka.findAll('tr'):
    
    ##REMOVE THE FIRST RECORD, SINCE IT IS DIFFERENT
    if (count == 0):
        count = count+1
    else:
        ##CUT ALL EXCEPT 'TD' DATA
        col = row.findAll('td')
        count = count+1
        records = []
        ##CLEAN 'TD' DATA AND ADD IT TO LIST
        records.append(col[0].string.strip())
        records.append(col[1].string.strip())
        records.append(col[2].text.strip())
        #record = '%s;%s;%s;' % (first, secound, third) # store the record with a ';' between prvy and druhy
        ##ADD RECORD TO LIST
        matrix.append(records)

In [6]:
## CREATE DATA FRAME ACCORDING TO MATRIX (LIST OF LIST)
df2 = pd.DataFrame(matrix, columns =['Postcode', 'Borough', 'Neighborhood'])
##df2.head()

### Filter Data

In [7]:
## FILTER DATA BOROUGH 1= NOT ASSIGNED
df = df2[df2.Borough != 'Not assigned']
## CHANGE NEIGHBORHOOD NAME IF IT COMES WITH A VALUE OF NOT ASSIGNED AS THE NAME OF THE BOROUGH
df.loc[df.Neighborhood == 'Not assigned', ['Neighborhood']] = df.Borough

C:\Users\MainUser\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\MainUser\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [8]:
## VALIDATE THAT IT IS NOT THERE
df.Borough.unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

### Aggregate Data

In [9]:
## GROUP AND ADD THE COMMA BETWEEN NEIGHBORS
df3 =[]
df3 = df.groupby('Postcode').agg({'Borough':'first',
                                'Neighborhood': ', '.join})
df3 = df3.sort_values(by=['Postcode'])

### See shape

In [10]:
df3.head()

,Borough,Neighborhood
Postcode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [11]:
df3.shape

(103, 2)

## Get Coordinates

In [12]:
path_csv = "C:/Users/MainUser/Downloads/Geospatial_Coordinates.csv"
##No header
df_2 = pd.read_csv(path_csv, header=0, index_col=0)

In [13]:
df_merge_difkey = pd.merge(df3, df_2, left_on=df3.index, right_on='Postal Code')
df3.index
df_merge_difkey.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
